# IPOTESI DI PRIMA BOZZA PER REGOLA 

Idea metto tutti gli attirbuti possibili di buildings in osm.
E poi ottengo i dataframe per ogni macroclasse con tutti i tipi di edificio e i numeri.
E poi scegliamo magari delle macrogateorie per raggruppare gli edifici secondo una classificazione nostra,che corrisponde poi a quella dei file di ramp. 
Tipo magari nel tagbuildign ci sono 4 tipi di edificio religioso con tag diverso, li riuniamo sotto un'unico tag church a cui corrisponde un file di modello di domanda.

In [ ]:
import pandas as pd

# Lettura del file Excel
building_type = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/number_buildings_type.xlsx"
)

# Dizionario dei tipi di edificio
type_dict = {
    "religious_type": [
        "religious",
        "cathedral",
        "chapel",
        "church",
        "kingdom_hall",
        "monastery",
        "mosque",
        "presbytery",
        "shrine",
        "synagogue",
        "temple",
    ],
    "accomodation_type": [
        "apartments",
        "barracks",
        "bungalow",
        "cabin",
        "detached",
        "dormitory",
        "farm",
        "ger",
        "hotel",
        "house",
        "houseboat",
        "residential",
        "semidetached_house",
        "static_caravan",
        "stilt_house",
        "terrace",
        "tree_house",
        "trullo",
    ],
    "commercial_type": [
        "commercial",
        "industrial",
        "kiosk",
        "office",
        "retail",
        "supermarket",
        "warehouse",
    ],
    "civic_type": [
        "bakehouse",
        "bridge",
        "civic",
        "college",
        "fire_station",
        "government",
        "gatehouse",
        "hospital",
        "kindergarten",
        "museum",
        "public",
        "school",
        "toilets",
        "train_station",
        "transportation",
        "university",
    ],
    "agriculture_type": [
        "barn",
        "conservatory",
        "cowshed",
        "farm_auxiliary",
        "greenhouse",
        "slurry_tank",
        "stable",
        "sty",
        "livestock",
    ],
    "sport_type": [
        "grandstand",
        "pavilion",
        "riding_hall",
        "sports_hall",
        "sports_centre",
        "stadium",
    ],
    "cars_type": ["carport", "garage", "garages", "parking"],
    "technical_type": [
        "digester",
        "service",
        "tech_cab",
        "transformer_tower",
        "water_tower",
        "storage_tank",
        "silo",
    ],
    "other_type": [
        "beach_hut",
        "bunker",
        "castle",
        "construction",
        "container",
        "guardhouse",
        "military",
        "outbuilding",
        "pagoda",
        "quonset_hut",
        "roof",
        "tent",
        "tower",
        "windmill",
        "yes",
        "user defined",
    ],
}

# Creazione dei DataFrame vuoti
dataframes = {}
for key, value in type_dict.items():
    df = pd.DataFrame(index=value)
    # Aggiunta delle colonne per i cluster
    for cluster in building_type["cluster"].unique():
        df[f"cluster_{cluster}"] = 0
    dataframes[key] = df


# Funzione per aggiornare i DataFrame per ogni cluster separatamente
def update_dataframes_by_cluster(main_df, dataframes):
    clusters = main_df["cluster"].unique()

    for cluster in clusters:
        cluster_df = main_df[main_df["cluster"] == cluster]

        for _, row in cluster_df.iterrows():
            for column in main_df.columns[1:]:  # Ignorare la colonna 'cluster'
                value = row[column]
                if value > 0:
                    for df in dataframes.values():
                        if column in df.index:
                            df.at[column, f"cluster_{cluster}"] += value


# Aggiornamento dei DataFrame per ogni cluster separatamente
update_dataframes_by_cluster(building_type, dataframes)

# Concatenazione dei DataFrame in un unico DataFrame per tipo di edificio
concatenated_dfs = {}
for key, df in dataframes.items():
    concatenated_dfs[key] = df
output_path = "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/"
for key, df in concatenated_dfs.items():
    df.to_csv(f"{output_path}_{key}.csv", index=True)

### REGOLA BUILD DEMAND CON RAMP:

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [2]:
tier_percent = [0.2, 0.2, 0.3, 0.2, 0.05, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,1570.0,1570.0,2355.0,1570.0,392.50,392.50
1,1859.4,1859.4,2789.1,1859.4,464.85,464.85
2,2214.2,2214.2,3321.3,2214.2,553.55,553.55
3,2192.0,2192.0,3288.0,2192.0,548.00,548.00
4,2317.6,2317.6,3476.4,2317.6,579.40,579.40
5,2371.6,2371.6,3557.4,2371.6,592.90,592.90
6,2171.0,2171.0,3256.5,2171.0,542.75,542.75
7,569.2,569.2,853.8,569.2,142.30,142.30
8,1847.0,1847.0,2770.5,1847.0,461.75,461.75


In [3]:
demand_tier_0 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Create a DataFrame of mean with all of the mean of each tier
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"mean_tier_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]

mean_demand_tier_df.insert(0, "mean_tier_0", np.zeros(len(mean_demand_tier_df)))
hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Create a DataFrame of std with all of the std of each tier
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"std_tier_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]

std_demand_tier_df.insert(0, "std_tier_0", np.zeros(len(mean_demand_tier_df)))
std_demand_tier_df.index = hours_index.time

C:\Users\margh\AppData\Local\Temp\ipykernel_24052\2397889867.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(start='00:00:00', periods=len(mean_demand_tier_df), freq='H', normalize=True)


In [165]:
dataframes_dict = {}

for index, row in tier_pop_df.iterrows():
    loads = []
    for col in range(tier_pop_df.shape[1]):
        n = row[col]
        tier = mean_demand_tier_df.iloc[:, col]
        loads_h = n * tier
        loads.append(loads_h)

    loads_df = pd.DataFrame(loads)
    loads_df = loads_df.T

    nome_dataframe = f"loads_cluster_{index}"
    dataframes_dict[nome_dataframe] = loads_df

display(dataframes_dict["loads_cluster_0"])

,mean_tier_0,mean_tier_1,mean_tier_2,mean_tier_3,mean_tier_4,mean_tier_5
00:00:00,0.0,0.000000,3470.298752,1920.767544,27252.224015,572.110603
01:00:00,0.0,0.000000,9655.147865,5222.030220,32835.946052,1558.049228
02:00:00,0.0,0.000000,13290.705357,7465.085308,34700.315289,2165.130691
03:00:00,0.0,0.000000,21192.291849,18105.092157,38626.725694,5606.142777
04:00:00,0.0,0.000000,51077.546685,57395.809933,51219.682273,18540.337622
05:00:00,0.0,0.000000,84181.964379,99012.919619,66407.655559,33850.663528
06:00:00,0.0,380.324702,75675.532234,98539.227337,75154.177285,36945.257575
07:00:00,0.0,7335.154291,54921.826508,64089.757630,85796.453952,33908.639046
08:00:00,0.0,565.090431,31038.909301,37540.501176,80072.810167,20196.940762
09:00:00,0.0,0.000000,12655.672113,19297.487981,72421.756224,8285.336919


In [51]:
tot_loads_dict = {}
for key in dataframes_dict:
    load = dataframes_dict[key]
    load_tot = load.sum(axis=1)

    total_loads_df = pd.DataFrame(load_tot, columns=[f"Total_{key}"])
    tot_loads_dict[key] = total_loads_df

loads_df = pd.DataFrame()
for key in tot_loads_dict:
    cluster_load = tot_loads_dict[key]
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

tot_loads_df.to_csv(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/demand/microgrid_load_2.csv"
)

NameError: name 'tot_loads_df' is not defined

In [210]:
# Creazione di un DataFrame vuoto per contenere i dati uniti
tot_loads_df = pd.DataFrame()

# Iterazione attraverso il dizionario dei DataFrame delle somme totali
for key in tot_loads_dict:
    cluster_load = tot_loads_dict[key]
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

daily_mean_profile_df = pd.DataFrame(tot_loads_df)
date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_profile_df = pd.concat(
    [daily_mean_profile_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_profile_df.index = date_time_index
yearly_profile_df.to_csv(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/demand/microgrid_load_2.csv"
)

C:\Users\margh\AppData\Local\Temp\ipykernel_19120\3745652416.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start='2013-01-01', end='2013-12-31 23:00:00', freq='H')


In [175]:
tot_loads_dict = {}
for key in dataframes_dict:
    load = dataframes_dict[key]
    load_tot = load.sum(axis=1)
    total_loads_df = pd.DataFrame(load_tot, columns=[f"Total_{key}"])
    name_dataframe = f"{key}"
    tot_loads_dict[nome_dataframe] = total_loads_df

tot_loads_dict

,Total_loads_cluster_0
00:00:00,33215.400913
01:00:00,49271.173366
02:00:00,57621.236645
03:00:00,83530.252477
04:00:00,178233.376513
05:00:00,283453.203085
06:00:00,286694.519132
07:00:00,246051.831427
08:00:00,169414.251836
09:00:00,112660.253238


In [32]:
cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/osm/raw/all_raw_building.geojson"
)
cleaned_buildings.rename(columns={"tags.building": "tags_building"}, inplace=True)
cleaned_buildings = cleaned_buildings.to_crs("ESRI:54009")
area_m2 = cleaned_buildings.geometry.area
area_m2 = pd.Series(area_m2)
cleaned_buildings["area_m2"] = area_m2
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_m2 = np.array(house["area_m2"])
percentile = np.percentile(area_m2, 95)
percentile

display(house)

,id,tags_building,building:levels,geometry,area_m2
636,667738440,house,None,"POLYGON ((734622.030 579069.511, 734628.858 57...",84.363195
637,667738441,house,None,"POLYGON ((734593.106 579071.314, 734602.927 57...",63.128585
638,667738442,house,None,"POLYGON ((734607.441 579076.500, 734606.954 57...",33.789995
639,667738443,house,None,"POLYGON ((734597.389 579057.495, 734600.785 57...",38.028242
640,667738444,house,None,"POLYGON ((734605.093 579053.087, 734607.691 57...",30.981288
...,...,...,...,...,...
15533,803749992,house,None,"POLYGON ((730165.695 578519.403, 730164.096 57...",180.566970
15648,804041367,house,None,"POLYGON ((730363.854 577681.297, 730370.641 57...",74.967738
15649,804041432,house,None,"POLYGON ((730346.873 577615.614, 730357.783 57...",226.334857
15650,804041811,house,None,"POLYGON ((730344.909 577516.466, 730335.077 57...",63.770662


### REGOLA BUILD DEMAND CON RAMP QUESTIONE DEVIAZIONE STANDARD.

In [22]:
demand_tier_0 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Create a DataFrame of mean with all of the mean of each tier
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"mean_tier_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]

mean_demand_tier_df.insert(0, "mean_tier_0", np.zeros(len(mean_demand_tier_df)))
hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Create a DataFrame of std with all of the std of each tier
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"std_tier_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]

std_demand_tier_df.insert(0, "std_tier_0", np.zeros(len(mean_demand_tier_df)))
std_demand_tier_df.index = hours_index.time

# Allungo i dataframe ristampandoli uguali per tutti i giorni dell'anno.
date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_mean_demand_tier_df = pd.concat(
    [mean_demand_tier_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_mean_demand_tier_df.index = date_time_index

yearly_std_demand_tier_df = pd.concat(
    [std_demand_tier_df] * len(date_range), ignore_index=True
)
yearly_std_demand_tier_df.index = date_time_index

display(yearly_mean_demand_tier_df)
yearly_std_demand_tier_df

C:\Users\margh\AppData\Local\Temp\ipykernel_24052\2592806886.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(start='00:00:00', periods=len(mean_demand_tier_df), freq='H', normalize=True)
C:\Users\margh\AppData\Local\Temp\ipykernel_24052\2592806886.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start='2013-01-01', end='2013-12-31 23:00:00', freq='H')


,mean_tier_0,mean_tier_1,mean_tier_2,mean_tier_3,mean_tier_4,mean_tier_5
2013-01-01 00:00:00,0.0,0.000000,1.522227,1.196115,69.661717,1.462838
2013-01-01 01:00:00,0.0,0.000000,4.056604,3.279885,83.855136,3.986523
2013-01-01 02:00:00,0.0,0.000000,5.699268,4.795340,88.435496,5.483456
2013-01-01 03:00:00,0.0,0.000000,9.040417,12.131634,98.316785,13.962898
2013-01-01 04:00:00,0.0,0.000000,21.776059,40.681523,129.949732,47.042389
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,47.652813,62.252508,58.018541,472.705325,117.554121
2013-12-31 20:00:00,0.0,55.958416,76.411150,84.322244,321.723879,151.491828
2013-12-31 21:00:00,0.0,36.647671,67.905680,93.383237,249.302548,149.749051
2013-12-31 22:00:00,0.0,19.730346,43.333531,76.202498,193.773689,112.843233


,std_tier_0,std_tier_1,std_tier_2,std_tier_3,std_tier_4,std_tier_5
2013-01-01 00:00:00,0.0,0.000000,0.429013,0.396138,2.178424,0.451707
2013-01-01 01:00:00,0.0,0.000000,0.810468,0.709363,2.339555,0.766178
2013-01-01 02:00:00,0.0,0.000000,0.942166,0.882295,2.547491,0.884457
2013-01-01 03:00:00,0.0,0.000000,1.360764,2.102167,3.699718,2.631140
2013-01-01 04:00:00,0.0,0.000000,2.758943,4.268914,6.274641,5.975277
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,2.462683,4.666371,4.926717,23.364815,8.475493
2013-12-31 20:00:00,0.0,2.446845,4.953632,5.474969,14.945005,8.145099
2013-12-31 21:00:00,0.0,2.218095,4.743251,5.180570,9.373338,8.283509
2013-12-31 22:00:00,0.0,2.049108,4.280691,4.721086,7.820964,7.995370


In [34]:
dataframes_dict = {}

for index, row in tier_pop_df.iterrows():
    loads = []
    for col in range(tier_pop_df.shape[1]):
        n = row[col]
        tier = yearly_std_demand_tier_df.iloc[:, col]
        loads_h = n * tier
        loads.append(loads_h)

    loads_df = pd.DataFrame(loads)
    loads_df = loads_df.T

    nome_dataframe = f"loads_cluster_{index}"
    dataframes_dict[nome_dataframe] = loads_df


dataframes_std_dict = {}

for index, row in tier_pop_df.iterrows():
    loads = []
    for col in range(tier_pop_df.shape[1]):
        n = np.sqrt(row[col])
        tier = yearly_std_demand_tier_df.iloc[:, col]
        loads_h = n * tier
        loads.append(loads_h)

    loads_df = pd.DataFrame(loads)
    loads_df = loads_df.T
    loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))

    nome_dataframe = f"std_cluster_{index}"
    dataframes_std_dict[nome_dataframe] = loads_df

display(dataframes_dict["loads_cluster_0"])
display(dataframes_std_dict["std_cluster_0"])

C:\Users\margh\AppData\Local\Temp\ipykernel_24052\303674484.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_24052\303674484.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_24052\303674484.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_24052\303674484.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_24052\303674484.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map inst

,std_tier_0,std_tier_1,std_tier_2,std_tier_3,std_tier_4,std_tier_5
2013-01-01 00:00:00,0.0,0.000000,1010.324694,621.937184,855.031534,177.294839
2013-01-01 01:00:00,0.0,0.000000,1908.651188,1113.699621,918.275328,300.724709
2013-01-01 02:00:00,0.0,0.000000,2218.801859,1385.203560,999.890247,347.149217
2013-01-01 03:00:00,0.0,0.000000,3204.599071,3300.401733,1452.139398,1032.722600
2013-01-01 04:00:00,0.0,0.000000,6497.310188,6702.194738,2462.796467,2345.296098
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,3866.412169,10989.304626,7734.945520,9170.689703,3326.631162
2013-12-31 20:00:00,0.0,3841.545868,11665.803713,8595.701246,5865.914358,3196.951256
2013-12-31 21:00:00,0.0,3482.409614,11170.356690,8133.494495,3679.035035,3251.277123
2013-12-31 22:00:00,0.0,3217.099141,10081.026157,7412.105695,3069.728482,3138.182651


,std_tier_0,std_tier_1,std_tier_2,std_tier_3,std_tier_4,std_tier_5
2013-01-01 00:00:00,0.0,0.000000,13.385388,0.030512,36.751722,-1.589021
2013-01-01 01:00:00,0.0,0.000000,47.661470,-76.158410,51.471956,-24.607167
2013-01-01 02:00:00,0.0,0.000000,25.792746,-9.333353,-13.387598,-4.461022
2013-01-01 03:00:00,0.0,0.000000,-12.941326,-50.242274,-15.020839,27.759474
2013-01-01 04:00:00,0.0,0.000000,-125.156076,100.858311,210.293182,31.748923
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,-106.750016,257.142730,-18.358547,-631.638105,94.912957
2013-12-31 20:00:00,0.0,-23.973461,311.199005,-276.506934,-221.070406,-22.933844
2013-12-31 21:00:00,0.0,111.254786,120.337274,-159.031892,262.431049,-137.382529
2013-12-31 22:00:00,0.0,-26.734561,-79.568896,-114.973119,252.405148,-106.484715


In [42]:
# Dizionario per memorizzare i dataframe risultanti
loads_with_std = {}

# Iterare sulle chiavi di uno dei due dizionari
for i in range(len(dataframes_dict)):
    key_df = f"loads_cluster_{i}"
    key_std = f"std_cluster_{i}"

    # Somma i dataframe corrispondenti
    if key_df in dataframes_dict and key_std in dataframes_std_dict:
        df = dataframes_dict[key_df] + dataframes_std_dict[key_std]
        loads_with_std[key_df] = df
    else:
        print(f"Keys {key_df} o {key_std} not in dictionary.")

loads_with_std["loads_cluster_0"]

,std_tier_0,std_tier_1,std_tier_2,std_tier_3,std_tier_4,std_tier_5
2013-01-01 00:00:00,0.0,0.000000,1023.710082,621.967696,891.783256,175.705818
2013-01-01 01:00:00,0.0,0.000000,1956.312658,1037.541211,969.747284,276.117542
2013-01-01 02:00:00,0.0,0.000000,2244.594605,1375.870206,986.502649,342.688195
2013-01-01 03:00:00,0.0,0.000000,3191.657745,3250.159459,1437.118559,1060.482074
2013-01-01 04:00:00,0.0,0.000000,6372.154112,6803.053049,2673.089649,2377.045020
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,3759.662153,11246.447357,7716.586973,8539.051599,3421.544119
2013-12-31 20:00:00,0.0,3817.572406,11977.002718,8319.194313,5644.843952,3174.017413
2013-12-31 21:00:00,0.0,3593.664399,11290.693964,7974.462603,3941.466084,3113.894593
2013-12-31 22:00:00,0.0,3190.364580,10001.457262,7297.132576,3322.133630,3031.697936


In [50]:
tot_loads_with_std_dict = {}
for key in loads_with_std:
    load = loads_with_std[key]
    load_tot = load.sum(axis=1)
    total_loads_df = pd.DataFrame(load_tot, columns=[f"Total_{key}"])
    # Assegna il dataframe totale al dizionario con il nome corretto
    tot_loads_with_std_dict[key] = total_loads_df

# Verifica del contenuto del dizionario
display(tot_loads_with_std_dict["loads_cluster_0"])

loads_df = pd.DataFrame()
for key in tot_loads_with_std_dict:
    cluster_load = tot_loads_with_std_dict[key]
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

,Total_loads_cluster_0
2013-01-01 00:00:00,2713.166853
2013-01-01 01:00:00,4239.718695
2013-01-01 02:00:00,4949.655655
2013-01-01 03:00:00,8939.417837
2013-01-01 04:00:00,18225.341831
...,...
2013-12-31 19:00:00,34683.292200
2013-12-31 20:00:00,32932.630802
2013-12-31 21:00:00,29914.181642
2013-12-31 22:00:00,26842.785983


NameError: name 'tot_loads_df' is not defined

In [1]:
tot_loads_dict = {}
for key in dataframes_dict:
    load = dataframes_dict[key]
    load_tot = load.sum(axis=1)

    total_loads_df = pd.DataFrame(load_tot, columns=[f"Total_{key}"])
    tot_loads_dict[key] = total_loads_df

loads_df = pd.DataFrame()
for key in tot_loads_dict:
    cluster_load = tot_loads_dict[key]
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

NameError: name 'dataframes_dict' is not defined

In [52]:
tot_loads_df = pd.DataFrame()
for key in tot_loads_with_std_dict:
    cluster_load = tot_loads_with_std_dict[key]
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

yearly_profile_df = pd.DataFrame(tot_loads_df)
yearly_profile_df.to_csv(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/demand/microgrid_load_3.csv"
)

# MODIFICHE ALLA REGOLA BUILD DEMAND: 

In [69]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [83]:
p = 1500

In [94]:
buildings_csv = pd.read_csv(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/number_buildings_type.csv"
)
total_buildings = buildings_csv.values.sum()
display(total_buildings)
buildings_for_cluster = []
for row in buildings_csv.itertuples(index=False, name=None):
    cluster_buildings = sum(row)
    buildings_for_cluster.append(cluster_buildings)

sum(buildings_for_cluster)
cluster_info_df = pd.DataFrame(buildings_for_cluster, columns=["Buildings"])
cluster_info_df = cluster_info_df
population_per_building = 10
cluster_info_df["population"] = cluster_info_df["Buildings"] * population_per_building
per_unit_load = (
    pd.read_csv(
        "C:/Users/margh/Desktop/pypsa/pypsa-distribution/data/sample_profile.csv"
    )["0"]
    / p
)
load_df = cluster_info_df["population"].apply(lambda x: x * per_unit_load)
new_index_names = [f"bus_{i}" for i in range(len(load_df))]
load_df = load_df.rename(index=dict(zip(load_df.index, new_index_names)))
load_df = load_df.T
display(load_df)

22078

,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9
0,225.440734,385.921872,307.599158,286.440889,335.933915,320.343612,325.540380,80.673633,251.053375,212.820013
1,250.163990,428.244503,341.332425,317.853807,372.774552,355.474517,361.241195,89.520813,278.585475,236.159200
2,175.517024,300.459713,239.481515,223.008732,261.541558,249.403718,253.449665,62.808507,195.457761,165.691153
3,200.418171,343.086869,273.457504,254.647676,298.647274,284.787401,289.407358,71.719345,223.187963,189.198273
4,162.644951,278.424590,221.918413,206.653711,242.360616,231.112941,234.862166,58.202255,181.123274,153.539690
...,...,...,...,...,...,...,...,...,...,...
8755,697.682992,1194.332191,951.942875,886.463297,1039.631901,991.383790,1007.466494,249.664825,776.947745,658.624998
8756,431.416148,738.521935,588.639157,548.149497,642.862153,613.027666,622.972495,154.381629,480.429948,407.264421
8757,403.435119,690.622468,550.460871,512.597311,601.167041,573.267576,582.567398,144.368660,449.269954,380.849838
8758,370.900838,634.928493,506.069969,471.259846,552.687033,527.037469,535.587324,132.726315,413.039407,350.136905


# CREAZIONE DEL FILE ALL_RAW_BUILDINGS PER IL TUTORIAL IN NIGERIA

In [ ]:
import osmium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon


class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.nodes = {}
        self.ways = []
        self.buildings = []

    def node(self, n):
        if n.location.valid():
            self.nodes[n.id] = (n.location.lon, n.location.lat)

    def way(self, w):
        if "building" in w.tags:
            nodes = []
            for node in w.nodes:
                if node.ref in self.nodes:
                    nodes.append(self.nodes[node.ref])
            if nodes:
                tags = {tag.k: tag.v for tag in w.tags if tag.k.startswith("building")}
                self.buildings.append([w.id, Polygon(nodes), tags])
        else:
            nodes = []
            for node in w.nodes:
                if node.ref in self.nodes:
                    nodes.append(self.nodes[node.ref])
            if nodes:
                self.ways.append([w.id, LineString(nodes)])


# Specifica il percorso al tuo file .osm
osm_file_path = "C:/Users/margh/Desktop/pypsa/pypsa-distribution/map.osm"

# Crea un handler per leggere il file OSM
handler = OSMHandler()

# Applica l'handler al file OSM
handler.apply_file(osm_file_path)

# Converti i nodi in un GeoDataFrame
nodes_gdf = gpd.GeoDataFrame(
    {
        "id": list(handler.nodes.keys()),
        "geometry": [Point(coord) for coord in handler.nodes.values()],
    }
)

# Converti le strade (ways) in un GeoDataFrame
ways_gdf = gpd.GeoDataFrame(handler.ways, columns=["id", "geometry"])

# Converti gli edifici in un GeoDataFrame
buildings_gdf = gpd.GeoDataFrame(handler.buildings, columns=["id", "geometry", "tags"])

# Aggiungi i tag come colonne separate nel GeoDataFrame
tags_df = buildings_gdf["tags"].apply(pd.Series)
buildings_gdf = buildings_gdf.join(tags_df).drop(columns=["tags"])
buildings_gdf.rename(columns={"building": "tags.building"}, inplace=True)
# Stampa i GeoDataFrame


# Salva il GeoDataFrame degli edifici in un file
buildings_gdf.to_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/osm/raw/all_raw_building.geojson",
    driver="GeoJSON",
)

In [ ]:
buildings_gdf.iloc[:, 2].unique()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os

buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/osm/raw/all_raw_building.geojson"
)
fig, ax = plt.subplots(figsize=(100, 70), subplot_kw={"projection": ccrs.PlateCarree()})
buildings.plot(
    ax=ax,
    legend=False,
    label="buildings",
    color="midnightblue",
    markersize=0.05,
    alpha=1,
)